In [1]:
import numpy as np

# Параметры для комбинаций
S_values = [2, 4]  # Число каналов
lambda_values = [1.2, 0.5]  # Интенсивность поступления
mu_values = [0.7, 0.3]  # Интенсивность обслуживания
n_values = [10, 20]  # Максимальное число заявок в системе
a, b = 1, 2  # Отрезок для генерации случайных чисел
num_iterations = 100  # Число итераций (обрабатываем 100 заявок)

# Функция для генерации времени события из равномерного распределения на [a, b]
def generate_exp_time(rate, a=1, b=2):
    u = np.random.uniform(a, b)  # Генерируем u на [1, 2]
    u_prime = (u - a) / (b - a)  # Преобразуем в [0, 1]
    return -np.log(1 - u_prime) / rate  # Экспоненциальное время

# Функция симуляции для заданных параметров
def simulate_queue(S, n, lambda_, mu, num_iterations):
    np.random.seed(42)  # Для воспроизводимости
    refusals = 0  # Счётчик отказов
    total_requests = 0  # Общее число заявок
    system_state_history = []  # История числа заявок в системе

    # Инициализация системы
    current_time = 0
    system_state = 0  # Текущее число заявок в системе
    servers_busy = 0  # Число занятых каналов
    queue = []  # Очередь времен завершения обслуживания

    # Генерируем время первого поступления
    next_arrival = generate_exp_time(lambda_)

    # Симуляция для num_iterations заявок
    for _ in range(num_iterations):
        # Определяем следующее событие: поступление или завершение обслуживания
        if not queue:
            next_service = float('inf')  # Нет заявок на обслуживании
        else:
            next_service = min(queue)  # Самое раннее завершение обслуживания

        if next_arrival <= next_service:
            # Событие: поступление заявки
            current_time = next_arrival
            total_requests += 1

            # Проверяем, есть ли место в системе
            if system_state < n:
                system_state += 1
                if servers_busy < S:
                    # Есть свободный канал
                    servers_busy += 1
                    service_time = generate_exp_time(mu)
                    queue.append(current_time + service_time)
                # Иначе заявка остаётся в очереди (system_state уже увеличен)
            else:
                refusals += 1  # Отказ

            # Генерируем время следующего поступления
            next_arrival = current_time + generate_exp_time(lambda_)
        else:
            # Событие: завершение обслуживания
            current_time = next_service
            queue.remove(next_service)
            servers_busy -= 1
            system_state -= 1

            # Если есть заявки в очереди, начинаем обслуживать следующую
            if system_state > servers_busy and servers_busy < S:
                servers_busy += 1
                service_time = generate_exp_time(mu)
                queue.append(current_time + service_time)

        system_state_history.append(system_state)

    # Статистика
    avg_system_state = np.mean(system_state_history)
    refusal_probability = refusals / total_requests
    return refusals, refusal_probability, avg_system_state

# Перебор всех комбинаций параметров
print("Симуляция СМО с отказом для различных комбинаций параметров")
print("S\tλ\tμ\tn\tЧисло отказов\tВероятность отказа\tСреднее число заявок")
print("-" * 80)

for S in S_values:
    for lambda_ in lambda_values:
        for mu in mu_values:
            for n in n_values:
                refusals, refusal_prob, avg_state = simulate_queue(S, n, lambda_, mu, num_iterations)
                print(f"{S}\t{lambda_}\t{mu}\t{n}\t{refusals}\t\t{refusal_prob:.3f}\t\t\t{avg_state:.3f}")

Симуляция СМО с отказом для различных комбинаций параметров
S	λ	μ	n	Число отказов	Вероятность отказа	Среднее число заявок
--------------------------------------------------------------------------------
2	1.2	0.7	10	0		0.000			2.780
2	1.2	0.7	20	0		0.000			2.780
2	1.2	0.3	10	37		0.507			8.880
2	1.2	0.3	20	27		0.370			16.950
2	0.5	0.7	10	0		0.000			1.720
2	0.5	0.7	20	0		0.000			1.720
2	0.5	0.3	10	0		0.000			3.260
2	0.5	0.3	20	0		0.000			3.260
4	1.2	0.7	10	0		0.000			2.060
4	1.2	0.7	20	0		0.000			2.060
4	1.2	0.3	10	13		0.213			8.300
4	1.2	0.3	20	3		0.049			14.830
4	0.5	0.7	10	0		0.000			1.360
4	0.5	0.7	20	0		0.000			1.360
4	0.5	0.3	10	0		0.000			2.020
4	0.5	0.3	20	0		0.000			2.020
